# wps_parameters

#### wps_parameters is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

In [2]:
docker_url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
url = os.environ.get('WPS_URL', docker_url)
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [3]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

Signature:
osprey.parameters(
    case_id,
    grid_id,
    pour_points=None,
    uh_box=None,
    routing=None,
    domain=None,
    loglevel='INFO',
    np=1,
    version=True,
    params_config_file=None,
    params_config_dict=None,
)
Docstring:
Develop impulse response functions using inputs from a configuration file or dictionary

Parameters
----------
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
case_id : string
    Case ID for the RVIC process (required)
grid_id : string
    Routing domain grid shortname (required)
pour_points : ComplexData:mimetype:`text/plain`, :mimetype:`text/csv`
    Path to Pour Points File; A comma separated file of outlets to route to [lons, lats] (required)
uh_box : ComplexData:mimetype:`text/plain`, :mimetype:`text/csv`
    Path to UH Box File. This defines the unit hydrograph to rout flow to the edge of each grid cell. (required)
routing : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`applica

#### We can use the docstring to ensure we provide the appropriate parameters.

In [4]:
# run parameters only with required inputs
output_basic = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/RVIC/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
)

In [5]:
# run parameters with local configuration file
output_config_file = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/RVIC/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
    params_config_file = resource_filename("tests", "/data/configs/parameters.cfg")
)

In [6]:
# run parameters with configuration dictionary
output_config_dict = osprey.parameters(
    case_id = "sample",
    grid_id = "COLUMBIA",
    pour_points = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/RVIC/sample_pour.txt",
    uh_box = resource_filename("tests", "/data/samples/uhbox.csv"),
    routing = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_flow_parameters.nc",
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
    params_config_dict = {
        "OPTIONS": {
            "LOG_LEVEL": "CRITICAL",
        },
    }
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [7]:
input_config = read_config("tests/data/configs/parameters.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d").zfill(2),
    "RvicPourPointsFile": "sample_pour.txt",
    "RvicFdrFile": "sample_flow_parameters.nc",
}

In [8]:
def test_parameters_output(output, expected):
    with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
        output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

        # Metadata of the output of RVIC parameters module
        metadata = {
            "title": output_data.title,
            "year": output_data.history.split()[5],
            "month": output_data.history.split()[2],
            "day": output_data.history.split()[3],
            "RvicPourPointsFile": output_data.RvicPourPointsFile,
            "RvicFdrFile": output_data.RvicFdrFile,
        }

        assert metadata == expected

In [9]:
test_parameters_output(output_basic, expected)
test_parameters_output(output_config_file, expected)
test_parameters_output(output_config_dict, expected)